 # Transformaciones Geométricas [ Recuperación de la Homografía ]
 Ingeniería en Datos e Inteligencia Organizacional
 
 Geometría Computacional ID0205 	
 
     Méndez Pool Joan de Jesús / 160300102

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

# Homografía


* Una homografía es una transformación lineal que mapea puntos de un plano con otros puntos del mismo plano. La homografía está definida por una matriz cuadrada, $H$, de orden tres.  
* Si $p=(x,y)$, son las coordenadas cartesianas de un punto, y $p_h=(x,y,1)$ las coordenadas homogéneas correspondientes, se determinan con el producto: 

$$Hp_h = p'$$


# <font color="red">Práctica</font>

En la siguiente celda, se genera un conjunto de puntos que después se transforma geométricamente a partir de una homografía conocida. El objetivo de la práctica es que recuperes los parámetros de la homografía utilizando mínimos cuadrados. Puedes utilizar la librería de $\texttt{numpy}$ para resolver las operaciones matriciales.

1. Elige tres parejas de puntos correspondientes, plantea dos sistemas de ecuaciones [cada uno de tres por tres] y resuélvelos para determinar los parámetros de la homografía.
2. Determina los parámetros de la homografía resolviendo el problema de mínimos cuadrados.
3. Compara las dos soluciones que obtuviste con la solución real ¿cuál se aproxima más?

In [2]:
n = 30
p = 10*(2*np.random.rand(n,2)-1)
H = np.array([[1.1, 0.2, 0], [-0.1, 0.98, 0], [0, 0.02, 1]])
o = np.ones((n,1))
ph = np.hstack((p,o))
pt = np.dot(ph, np.transpose(H))

In [3]:
# Funciones para resolver Sistemas de Ecuaciones Lineales
def RecuperaHomografiaCramer(p, pt):
# Aplicamos el método de Cramer que nos permite solucionar Sistemas de Ecuaciones Lineales
# compatibles determinados (con una única solución) mediante el cálculo de determinantes.

    # Definimos nuestro Sistema de Ecuaciones Lineales como una Matriz de Coeficientes 'A'
    A = np.array( [[p[0,0], p[0,1], 0], [p[1,0], p[1,1], 0], [p[2,0], p[2,1], 1]] )
    # Calculamos el Determinante de 'A'
    delta = np.linalg.det(A)
    # Creamos nuestra lista de arreglos de la Homografía
    h = []
    # Realizamos un for para la columna X' e Y'
    for i in range(2):
        # Lista de los resultados del cálculo de las incógnitas ( [x, y, z] )
        l = []
        # La regla de Cramer nos dice que debemos cambiar la columna de la incógnita actual 
        # desde 'x' a 'z' ([x,y,z]) por la matriz de los términos independientes (ósea X', Y')
        for j in range(3):
            a = A.copy()
            # Sustituimos los valores de la columna de la incógnita actual
            a[:,j] = pt[:3,i]
            # Realizamos el cálculo de la incógnita y la guardamos en la lista
            l.append(np.linalg.det(a)/delta)
        # Guardamos la lista obtenida de la aplicación del método de Cramer
        h.append(l)
    # Acompletamos la matriz de Homografía con los términos que ya conocemos [0,0,1]
    h.append([0,0,1])
    # Redondeamos los valores obtenidos de la matriz de incógnitas a dos decimales
    return np.around(np.array(h), decimals=2)

def RecuperaHomografiaNumpySolve(p, pt):
# Usando la librería numpy con el módulo de Álgebra Lineal, el método 'Solucionar' nos permite
# resolver sistemas de ecuaciones a partir de la matriz de coeficientes y
# la matriz de términos independientes.

    # Definimos nuestro Sistema de Ecuaciones Lineales (sel) con la matriz de coeficientes
    sel = np.array( [[p[0,0], p[0,1], 0], [p[1,0], p[1,1], 0], [p[2,0], p[2,1], 1] ] )
    # Resolvemos para la columna X'
    a1 = np.around(np.linalg.solve(sel, pt[:3,0]), decimals=2)
    # Resolvemos para la columna  Y'
    a2 = np.around(np.linalg.solve(sel, pt[:3,1]), decimals=2)
    # Acompletamos la matriz con los términos ya conocidos de la homografía [0,0,1]
    a3 = np.zeros(3)
    a3[-1]=1
    return np.array([a1, a2, a3])

# Funciones para resolver Sistemas de Ecuaciones Lineales por Mínimos Cuadrados
def MinimosCuadrados(p, pt):
# Usando Cálculo Diferencial, mediante operaciones matriciales
# podemos obtener la tripleta que resuelve el problema de Mínimos Cuadrados
# con la formula definida anteriormente:
    # 𝐚1=(𝑃^(T) x 𝑃)⁻¹ x (𝑃^(𝑇) x 𝑋′)
    
    # Primero acompletamos la matriz de coeficientes con unos
    ph = np.hstack((p,np.ones((len(p),1))))
    # Realizamos el cálculo de la matriz inversa del producto punto de 'ph' por su traspuesta
    pinv = np.linalg.inv(np.dot(np.transpose(ph),ph))
    # Realizamos producto punto de la matriz traspuesta de 'ph' por la matriz de términos independientes X' e Y'
    pTti = np.dot(np.transpose(ph), pt)
    # Transponemos el resultado del producto punto de los dos cálculos anteriores 
    mc = np.transpose(np.dot(pinv,pTti))
    # Redondeamos los valores obtenidos de la matriz de Homografía a dos decimales
    return np.around(mc, decimals=2)

def MinimosCuadradosNumpyLeastSquares(p, pt):
# Usando la librería numpy con el módulo de Álgebra Lineal, el método 'Mínimos Cuadrados' nos permite
# obtener los valores mínimos de la matriz de Homografía

    # Primero acompletamos la matriz de coeficientes con unos
    ph = np.hstack((p,np.ones((len(p),1))))
    # Resolvemos para la columna X'
    a1 = np.around(np.linalg.lstsq(ph,pt[:,0], rcond=None)[0], decimals=2)
    # Resolvemos para la columna Y'
    a2 = np.around(np.linalg.lstsq(ph,pt[:,1], rcond=None)[0], decimals=2)
    # Resolvemos para la columna de unos
    a3 = np.around(np.linalg.lstsq(ph,pt[:,2], rcond=None)[0], decimals=2)
    return np.array([a1,a2,a3])

def RecuperaHomografiaOpenCV(p, pt):
# Usando la librería OpenCV, el método 'Encuentra Homografía'
# Nos retorna en el primer arreglo del resultado los valores de la matriz de Homografía
# que transformó a los puntos originales
    
    # Forma Simple:
    #print np.around(np.transpose(cv2.findHomography(p, pt))[0], decimals=2)
    
    # Obtenemos la matriz de Homografía
    h = cv2.findHomography(p, pt)[0]
    # Redondeamos los valores de la matriz a dos decimales
    h = np.around(h, decimals=2)
    return h

def MetodosdeRecuperaciondeHomografia(p, pt, H):
    print "Recuperación de la Homografía\n"
    print "\nHomografía Original:\n" + str(H)
    # Resolución a Sistemas de Ecuaciones
    print '\nUsando Método de Cramer:\n\n' + str(RecuperaHomografiaCramer(p, pt))
    print '\nUsando Numpy.linalg.solve():\n\n' + str(RecuperaHomografiaNumpySolve(p, pt))
    # Método de Mínimos Cuadrados
    print '\nUsando Mínimos Cuadrados:\n\n' + str(MinimosCuadrados(p, pt))
    print '\nUsando Numpy.linalg.lstsq():\n\n' + str(MinimosCuadradosNumpyLeastSquares(p, pt))
    print '\nUsando OpenCV.findHomography():\n\n' + str(RecuperaHomografiaOpenCV(p, pt))

Se llegarón a componer diferentes rutinas para la solución de este problema, teniendo como tal dos que se resuelven por Sistema de Ecuaciones Lineales, uno con el método de Cramer y otro con la librería de <i>Numpy</i> por el otro lado tenemos tres que efectuan el Cálculo de los Mínimos Cuadrados, uno compuesto por operaciones matriciales, y los otros dos compuestos con ayuda de las librerías <i>Numpy</i> y <i>OpevCV</i>. Por lo que a contrinuación tenemos las pruebas de las diferentes rutinas planteadas:

In [4]:
MetodosdeRecuperaciondeHomografia(p, pt, H)

Recuperación de la Homografía


Homografía Original:
[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.02  1.  ]]

Usando Método de Cramer:

[[ 1.1   0.2   0.  ]
 [-0.1   0.98 -0.  ]
 [ 0.    0.    1.  ]]

Usando Numpy.linalg.solve():

[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.    1.  ]]

Usando Mínimos Cuadrados:

[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.02  1.  ]]

Usando Numpy.linalg.lstsq():

[[ 1.1   0.2  -0.  ]
 [-0.1   0.98 -0.  ]
 [-0.    0.02  1.  ]]

Usando OpenCV.findHomography():

[[ 1.1   0.2  -0.  ]
 [-0.1   0.98  0.  ]
 [-0.    0.02  1.  ]]


## Matriz de Homografía Original

Tenemos dos soluciones para el mismo problema, por lo cual debemos evaluar cual solución se aproxima más a los valores originales que componen la matriz de Homografía:

<ul type='a'>
    <li>Dado que debemos obtener los valores de la Homografía resolviendo el Sistema de Ecuaciones Lineales puede llegar a ser el primer método para solucionar la problemática, nuestro sistema de ecuaciones planteado es de <i>Nx3</i>, tenemos un Sistema de Ecuaciones Rectangular, esto quiere decir que nuestro número de ecuaciones es mayor a nuestro número de incógnitas por lo que podemos decir que tenemos información de más a la hora de hacer la evaluación de las incógnitas, por lo que debemos solo utilizar el mismo número de ecuaciones e incógnitas para resolver el Sistema de Ecuaciones.</li>
    <li> Para resolver por el segundo Método se lleva a cabo operaciones matriciales que realizamos con el método de Mínimos Cuadrados para obtener los valores de la matriz de Homografía que resuelva todo el Sistema de Ecuaciones Lineales. Esto se lleva a cabo cálculando el margen de errores para cada ecuación.
    </li>
</ul>

### Solución al Sistema de Ecuaciones por el Método de Cramer

Abscisa:

\begin{align}
    x'_1 &= a_{11}x_1+a_{12}y_1+a_{13}\\
    x'_2 &= a_{11}x_2+a_{12}y_2+a_{13}\\
    \vdots &= \vdots \\
    x'_n &= a_{11}x_n+a_{12}y_n+a_{13}\\
\end{align}

Ordenada:

\begin{align}
    y'_1 &= a_{21}x_1+a_{22}y_1+a_{23}\\
    y'_2 &= a_{21}x_2+a_{22}y_2+a_{23}\\
    \vdots &= \vdots \\
    y'_n &= a_{21}x_n+a_{22}y_n+a_{23}\\
\end{align}

In [5]:
print "Homografía Original: \n\n" + str(H)
print '\nUsando Método de Cramer:\n\n' + str(RecuperaHomografiaCramer(p, pt))

Homografía Original: 

[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.02  1.  ]]

Usando Método de Cramer:

[[ 1.1   0.2   0.  ]
 [-0.1   0.98 -0.  ]
 [ 0.    0.    1.  ]]


Observamos que las primeras dos filas de la matriz de Homografía obtenida coinciden con la Homografía Original
pero en la tercera se pierde la presición con el valor valor de la matriz en la posición H<sub>[2][0]</sub>, ya que al realizar la operación con la columna llena de 'unos' de la matriz <i>Q</i> se obtienen resultados que varían mucho en comparación con la matriz de Homografía Original, esto se resuelve de forma sencilla porque los valores de la tercera fila ya son conocidos con anterioridad antes de realizar las operaciones con las matrices. 

### Solución al Sistema de Ecuaciones por el Método de Mínimos Cuadrados

Con este método debemos definir el margen de errores. 
\begin{align}
    e_1 &= x'_1-(a_{11}x_1+a_{12}y_1+a_{13})\\
    e_2 &= x'_2-(a_{11}x_2+a_{12}y_1+a_{13})\\
    \vdots &= \vdots\\
    e_n &= x'_n-(a_{11}x_n+a_{12}y_n+a_{13})\\
\end{align}

Fórmula aplicable para la resolución de este problema:
$$\mathbf{a_1} = (P^{T}P)^{-1}P^{T}X'$$

In [6]:
print "Homografía Original: \n\n" + str(H)
print '\nUsando Mínimos Cuadrados:\n\n' + str(MinimosCuadrados(p, pt))

Homografía Original: 

[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.02  1.  ]]

Usando Mínimos Cuadrados:

[[ 1.1   0.2   0.  ]
 [-0.1   0.98  0.  ]
 [ 0.    0.02  1.  ]]


Ahora podemos observar que la precisión de este Método es mejor que el usado anteriormente ya que nos devuelve todos los valores correspondientes exactamente iguales a la Homografía Original variando unicamente en el <i>-0</i> pero es equivalente ya que matemáticamente no existe como tal dicho número ( es igual a zero ), por lo que realizando el cálculo de errores podemos obtener de forma más precisa la solución con ayuda de este método, aunque afortunadamente tenemos la fórmula simplificada para realizar las operaciones matriciales que componen la solución con este método que gracias a las librerías de <i>Python</i> se pueden realizar efectivamente con rutinas ya programadas.

### Conclusión

Al comparar los dos métodos tenemos que es más eficiente utilizar los mínimos cuadrados ya que al calcular el margen de errores se puede obtener los valores mínimos que se necesitan para cumplir la solución de las incógnitas a los Sistemas de Ecuaciones Lineales plateados como matrices de Coeficientes, por lo tal tenemos que el método de los Mínimos Cuadrados es más utilizado en las aplicaciones de este método por su precisión, por lo que diferentes librerías como <i>Scypy</i>, <i>Numpy</i> y <i>OpenCV</i> tienen en sus módulos rutinas que utilizan dicho método para realizar los cálculos correspondientes en cuanto a Homografías se refiere.